In [1]:
import pandas as pd
from pipedrive import Pipedrive
import json
try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode

# pipedrive = Pipedrive('06395da8df833c411f231a703d5df4ca28809d87')
pipedrive = Pipedrive("51c436623dcf240bfc2b59e08100db09f7af9310")

In [2]:
# pull all deals in abacus 
more_deals = True
deal_lists = []
new_start = 0

while more_deals:
    print(new_start)
    pipedrive_deals = pipedrive.deals({'status':'all_not_deleted', 'limit': 500, 'start': new_start}, method='GET')
    new_deals = pipedrive_deals['data']
    deal_dataframe = pd.DataFrame(new_deals)
    deal_lists.append(deal_dataframe)
    print(len(deal_lists))
    more_deals = pipedrive_deals['additional_data']['pagination']['more_items_in_collection']
    new_start += 500

df = pd.concat(deal_lists)

0
1
500
2


In [3]:
stages_df = pd.DataFrame(pipedrive.stages({'limit': 500}, method='GET')['data'])

In [4]:
stages_df

,active_flag,add_time,deal_probability,id,name,order_nr,pipeline_id,pipeline_name,rotten_days,rotten_flag,update_time
0,True,2018-04-10 12:26:34,0,18,Hold,1,1,Pipeline,None,False,2018-05-17 21:15:59
1,True,2016-12-19 14:29:12,0,9,Qualified Leads to be Worked,2,1,Pipeline,None,False,2018-05-17 21:16:04
2,True,2017-12-28 19:25:37,0,10,Qualified Leads Being Worked,3,1,Pipeline,None,False,2018-05-17 21:16:09
3,True,2018-03-08 17:43:48,0,12,Adam Emails,4,1,Pipeline,None,False,2018-05-17 21:16:16
4,True,2016-11-04 11:26:50,10,2,Contact Made & Interested,5,1,Pipeline,None,False,2018-06-14 11:42:45
5,True,2018-03-20 20:40:23,30,17,Demo Scheduled,6,1,Pipeline,None,False,2018-06-14 11:42:56
6,True,2016-11-04 11:26:50,40,3,Demo Completed,7,1,Pipeline,None,False,2018-06-14 11:43:11
7,True,2018-03-09 15:45:34,40,14,Multiple Demos,8,1,Pipeline,None,False,2018-05-17 21:11:35
8,True,2016-11-04 11:26:50,60,4,Proposal Made,9,1,Pipeline,None,False,2018-06-14 11:43:34
9,True,2016-11-04 11:26:50,75,5,Negotiations,10,1,Pipeline,None,False,2018-06-14 11:43:51


In [5]:
# domains
def extract_domain(person_id):
    try:
        email = person_id['email'][0]['value']
        if "@" in email:
            return email.split("@")[1]
    except:
        print(person_id)

df['domain'] = df.person_id.apply(extract_domain)

None
None


In [15]:
df[df.title == "Villa Healthcare deal"]

,a57908157b42cc601c33f0db7582f0dc87ef39c2,active,activities_count,add_time,cc_email,close_time,creator_user_id,currency,deleted,done_activities_count,...,title,undone_activities_count,update_time,user_id,value,visible_to,weighted_value,weighted_value_currency,won_time,domain
394,3,True,0,2018-07-30 14:48:43,apploi+deal1564@pipedrivemail.com,None,"{'id': 1874156, 'name': 'Adam Lewis', 'email':...",USD,False,0,...,Villa Healthcare deal,0,2018-10-05 16:25:42,"{'id': 1874156, 'name': 'Adam Lewis', 'email':...",3000,3,3000,USD,None,villahc.com


In [14]:
print(len(df[df.domain.isnull()]))
len(df)

318


521

In [12]:
df[df.title == 'Starbucks deal'].person_id[0]

{'name': 'Afsheen Saatchi',
 'email': [{'label': 'work',
   'value': 'asaatchi@starbucks.com',
   'primary': True}],
 'phone': [{'label': 'work', 'value': '2063185572', 'primary': True}],
 'value': 1}

In [119]:
def extract_domain(Email):
    return Email.split("www.")[1]

pr_df = pd.read_csv("Apploi-linkedin.csv")
pr_df['domain'] = pr_df[pr_df.Website.notnull()].Website.apply(extract_domain)
pr_domains = pr_df.domain.tolist()

# n_df = pd.read_csv('apollo-neutral.csv')
# n_df['domain'] = n_df.Email.apply(extract_domain)
# n_domains = n_df.domain.tolist()

# ni_df = pd.read_csv("apollo-not-interested.csv")
# ni_df['domain'] = ni_df.Email.apply(extract_domain)
# ni_domains = ni_df.domain.tolist()

In [120]:
df = df[(df.domain.isin(pr_domains))]

In [121]:
def get_stage_prop(stage_id):
    return stages_df[stages_df.id == stage_id].iloc[0].deal_probability

df['stage_prob'] = df.stage_id.apply(get_stage_prop)

def get_stage_name(stage_id):
    return stages_df[stages_df.id == stage_id].iloc[0].name

df['stage_name'] = df.stage_id.apply(get_stage_name)

In [122]:
df.columns.values

array(['a57908157b42cc601c33f0db7582f0dc87ef39c2', 'active',
       'activities_count', 'add_time', 'cc_email', 'close_time',
       'creator_user_id', 'currency', 'deleted', 'done_activities_count',
       'email_messages_count', 'expected_close_date', 'files_count',
       'first_won_time', 'followers_count', 'formatted_value',
       'formatted_weighted_value', 'id', 'last_activity_date',
       'last_activity_id', 'last_incoming_mail_time',
       'last_outgoing_mail_time', 'lost_reason', 'lost_time',
       'next_activity_date', 'next_activity_duration', 'next_activity_id',
       'next_activity_note', 'next_activity_subject',
       'next_activity_time', 'next_activity_type', 'notes_count',
       'org_hidden', 'org_id', 'org_name', 'owner_name',
       'participants_count', 'person_hidden', 'person_id', 'person_name',
       'pipeline_id', 'probability', 'products_count',
       'reference_activities_count', 'rotten_time', 'stage_change_time',
       'stage_id', 'stage_order_nr'

In [123]:
# for index, deal in df.iterrows():
#     deals.insert({
#         'title': deal['title'],
#         'value': deal['value'],
#         'channel_id': "zb5TcJ9GjLzrWhSY3p",
#         'stage_name': deal['stage_name'],
#         'stage_prob': deal['stage_prob'] / 100,
#         'expected_value': deal['value'] * deal['stage_prob'] / 100,
#         "activities_count": deal['activities_count'],
#         "add_time": deal['add_time']
#     })

In [124]:
# deal break down
df.status.value_counts()

open    2
Name: status, dtype: int64

In [125]:
# # exclude all agency partner deals - should we still get credit for lost deals? (I personally dont think we should)
# df = df[~(df.status == 'lost')]
# len(df)

In [126]:
df.title

297    Brandywine Living deal
419    Amada Senior Care deal
Name: title, dtype: object

In [113]:
# break down of deal value
df.value.value_counts()

Series([], Name: value, dtype: int64)

In [108]:
# deals with value
df = df[~(df.value == 0)]
len(df)

0

In [109]:
df.columns.values

array(['a57908157b42cc601c33f0db7582f0dc87ef39c2', 'active',
       'activities_count', 'add_time', 'cc_email', 'close_time',
       'creator_user_id', 'currency', 'deleted', 'done_activities_count',
       'email_messages_count', 'expected_close_date', 'files_count',
       'first_won_time', 'followers_count', 'formatted_value',
       'formatted_weighted_value', 'id', 'last_activity_date',
       'last_activity_id', 'last_incoming_mail_time',
       'last_outgoing_mail_time', 'lost_reason', 'lost_time',
       'next_activity_date', 'next_activity_duration', 'next_activity_id',
       'next_activity_note', 'next_activity_subject',
       'next_activity_time', 'next_activity_type', 'notes_count',
       'org_hidden', 'org_id', 'org_name', 'owner_name',
       'participants_count', 'person_hidden', 'person_id', 'person_name',
       'pipeline_id', 'probability', 'products_count',
       'reference_activities_count', 'rotten_time', 'stage_change_time',
       'stage_id', 'stage_order_nr'

In [110]:
# deal statistics
df.value.describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: value, dtype: float64

In [111]:
# how many deals are in each stage
df.stage_id.value_counts()

Series([], Name: stage_id, dtype: int64)

In [21]:
stages = pd.DataFrame(pipedrive.stages({'limit': 500}, method='GET')['data'])
stages

,active_flag,add_time,deal_probability,id,name,order_nr,pipeline_id,pipeline_name,rotten_days,rotten_flag,update_time
0,True,2018-04-10 12:26:34,0,18,Hold,1,1,Pipeline,None,False,2018-05-17 21:15:59
1,True,2016-12-19 14:29:12,0,9,Qualified Leads to be Worked,2,1,Pipeline,None,False,2018-05-17 21:16:04
2,True,2017-12-28 19:25:37,0,10,Qualified Leads Being Worked,3,1,Pipeline,None,False,2018-05-17 21:16:09
3,True,2018-03-08 17:43:48,0,12,Adam Emails,4,1,Pipeline,None,False,2018-05-17 21:16:16
4,True,2016-11-04 11:26:50,10,2,Contact Made & Interested,5,1,Pipeline,None,False,2018-06-14 11:42:45
5,True,2018-03-20 20:40:23,30,17,Demo Scheduled,6,1,Pipeline,None,False,2018-06-14 11:42:56
6,True,2016-11-04 11:26:50,40,3,Demo Completed,7,1,Pipeline,None,False,2018-06-14 11:43:11
7,True,2018-03-09 15:45:34,40,14,Multiple Demos,8,1,Pipeline,None,False,2018-05-17 21:11:35
8,True,2016-11-04 11:26:50,60,4,Proposal Made,9,1,Pipeline,None,False,2018-06-14 11:43:34
9,True,2016-11-04 11:26:50,75,5,Negotiations,10,1,Pipeline,None,False,2018-06-14 11:43:51


In [22]:
# create proability of closing column
def expected_value(stage_id):
    return stages[stages.id == stage_id].iloc[0].deal_probability / 100

df['deal_probability'] = df.stage_id.apply(expected_value)

In [23]:
# calc expected value for each deal ( probability of closing * deal value)
df['expected_value'] = df.value * df.deal_probability

In [24]:
# total pipeline for Abacus
df.expected_value.sum()

9500.0

In [25]:
# remove all deals with no expected value
df = df[~(df.expected_value == 0)]

In [55]:
# total revenue received
df[df.stage_id == 15].expected_value.sum()

3500.0

In [27]:
# revenue breakdown
df[(df.status == 'won')].expected_value.describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: expected_value, dtype: float64

In [28]:
# expected revenue from open deals
df[df.status == 'open'].expected_value.sum()

3500.0

In [56]:
# expect revenue break down
df[(df.status == 'open')].expected_value.describe()

count       2.000000
mean     1750.000000
std      1767.766953
min       500.000000
25%      1125.000000
50%      1750.000000
75%      2375.000000
max      3000.000000
Name: expected_value, dtype: float64

In [57]:
def extract_domain(person_id):
    try:
        email = person_id['email'][0]['value']
        if "@" in email:
            return email.split("@")[1]
    except:
        print(person_id)

df['domain'] = df.person_id.apply(extract_domain)


def extract_email(person_id):
    try:
        return person_id['email'][0]['value']
    except:
        print(person_id)

df['email'] = df.person_id.apply(extract_email)

In [58]:
df.domain

2       meijer.com
394    villahc.com
413     phcpdn.com
Name: domain, dtype: object

In [59]:
# only include deals that have a domain
df = df[df.email.notnull()]
len(df)

3

In [60]:
# PR emails GG has generated
pr_df = pd.read_csv("apollo-interested.csv")
pr_emails = pr_df.Email.tolist()
len(pr_emails)

50

In [61]:
# pipeline deals we've sent them from prs
pipeline_df = df[df.email.isin(pr_emails)]
len(pipeline_df)

1

In [62]:
# total pipeline deals we've generate from prs
pipeline_df.expected_value.sum()

500.0

In [64]:
# prs closed from GG
renvue_df = pipeline_df[pipeline_df.stage_id == 15]
len(renvue_df)

1

In [65]:
# total revenue they have closed from GG
renvue_df.value.sum()

500

In [66]:
# deal name
renvue_df.domain

413    phcpdn.com
Name: domain, dtype: object

In [67]:
# do the same analysis by name
def extract_domain(Email):
    return Email.split("@")[1]

pr_df['domain'] = pr_df.Email.apply(extract_domain)
pr_domains = pr_df.domain.tolist()

In [68]:
# interested number of deals by domain
domain_pipeline_df = df[df.domain.isin(pr_domains)]
len(domain_pipeline_df)

2

In [69]:
# expected for positive replies by domain
domain_pipeline_df.expected_value.sum()

3500.0

In [71]:
# how many deals have we won by domain
domain_revenue_df = domain_pipeline_df[domain_pipeline_df.stage_id == 15]
len(domain_revenue_df)

2

In [72]:
domain_revenue_df.value.sum()

3500

In [73]:
# emails of those who indicated interested
# lcherry@fool.com is her actual email that is spelt different in the abacus CRM than it is in Apollo
domain_revenue_df.email

394    mtatelbame@villahc.com
413          laura@phcpdn.com
Name: email, dtype: object

In [74]:
################ NEUTRAL #########################

In [75]:
# neutral leads GG has generated
n_df = pd.read_csv('apollo-neutral.csv')
len(n_df)

45

In [76]:
# pipeline deals we've sent them from neutrals
pipeline_df = df[df.email.isin(n_df.Email.tolist())]
len(pipeline_df)

0

In [77]:
pipeline_df.expected_value.sum()

0.0

In [78]:
pipeline_df.domain

Series([], Name: domain, dtype: object)

In [79]:
def extract_domain(Email):
    return Email.split("@")[1]

n_df['domain'] = n_df.Email.apply(extract_domain)
n_domains = n_df.domain.tolist()

In [80]:
# neutral number of deals
neutral_df = df[df.domain.isin(n_domains)]
len(neutral_df)

0

In [81]:
# total pipeline for neutral replies
neutral_df.expected_value.sum()

0.0

In [82]:
# titles of deals who indicate neutral
# make sure to look up tophatter.com in Apollo - it's a referral
neutral_df.domain

Series([], Name: domain, dtype: object)

In [54]:
# total won deals from neutral
won_neutral = neutral_df[neutral_df.status == 'won']
len(won_neutral)

0

In [55]:
################ NOT INTERESTED #########################

In [83]:
# total not interested
ni_df = pd.read_csv("not-interested.csv")
len(ni_df)

228

In [84]:
# pipeline deals we've sent them from not interested
pipeline_df = df[df.email.isin(ni_df.Email.tolist())]
len(pipeline_df)

0

In [85]:
pipeline_df.formatted_value

Series([], Name: formatted_value, dtype: object)

In [59]:
pipeline_df.email

276           tgiles@commvault.com
405                   ian@lens.com
325           jcamden@spisales.com
22     scott.spence@fundsquire.com
Name: email, dtype: object

In [60]:
def extract_domain(Email):
    return Email.split("@")[1]

ni_df['domain'] = ni_df.Email.apply(extract_domain)
ni_domains = ni_df.domain.tolist()

In [61]:
# neutral number of deals
len(df[df.domain.isin(ni_domains)])

14

In [62]:
# total pipeline for not interested replies
df[df.domain.isin(ni_domains)].expected_value.sum()

95000.0

In [63]:
#  domains of deals who indicate not interested
df[df.domain.isin(ni_domains)].domain

96      eddiebauer.com
211       sothebys.com
233     outsystems.com
269        tradesy.com
276      commvault.com
279           hulu.com
306         forbes.com
320         hiscox.com
405           lens.com
324    selectquote.com
325       spisales.com
475         kbhome.com
22      fundsquire.com
35            uncg.edu
Name: domain, dtype: object

In [64]:
df[df.domain == 'fundsquire.com'].formatted_value

22    $30,000
Name: formatted_value, dtype: object

In [65]:
################ LINKEDIN #########################

In [66]:
# total linkedin
li_df = pd.read_csv('linkedin.csv')
len(li_df)

1129

In [67]:
# pipeline deals we've sent them from neutrals
pipeline_df = df[df.email.isin(li_df.Email.tolist())]
len(pipeline_df)

0

In [68]:
pipeline_df.formatted_value

Series([], Name: formatted_value, dtype: object)

In [69]:
# harrison entered their funnel outside of GG - checked Peter's linkedin (should check lances as well)
pipeline_df.email

Series([], Name: email, dtype: object)

In [70]:
def extract_domain(Email):
    try:
        return Email.split("@")[1]
    except:
        return ''

li_df['domain'] = li_df.Email.apply(extract_domain)
li_domains = li_df.domain.tolist()

In [71]:
# neutral number of event deals
len(df[df.domain.isin(li_domains)])

0

In [72]:
# total pipeline for not interested replies
df[df.domain.isin(li_domains)].expected_value.sum()

0.0

In [73]:
# titles of deals who indicate not interested
df[df.domain.isin(event_domains)].domain

NameError: name 'event_domains' is not defined

In [ ]:
df[df.domain == '1milk2sugarspr.com']

In [ ]:
# need to login as peter to finish this attribution analysis for events

In [74]:
################ TOTAL ROI #########################

In [75]:
# total GG deals
len(df[df.domain.isin(n_domains)]) + len(df[df.domain.isin(ni_domains)]) + len(df[df.domain.isin(pr_domains)]) + len(df[df.domain.isin(li_domains)])

90

In [76]:
gg_total = df[df.domain.isin(ni_domains)].expected_value.sum() + df[df.domain.isin(n_domains)].expected_value.sum() + df[df.domain.isin(pr_domains)].expected_value.sum() + df[df.domain.isin(li_domains)].expected_value.sum()

In [77]:
gg_total

763300.0

In [78]:
gg_total / df.expected_value.sum() * 100

118.19448745741717